# Implicit Feedback Recommender



## 1. Setup and Imports

In [21]:
from itertools import product
from pathlib import Path
import pickle
from typing import Literal

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import leave_k_out_split
from implicit.nearest_neighbours import bm25_weight
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix
#from tqdm.notebook import trange, tqdm_notebook
from tqdm import trange, tqdm


In [22]:
RANDOM_SEED = 42


## 2. Data Loading

Define functions for data loading and preprocessing.

In [23]:
EVENT_STRENGTH: dict[str, float] = {
    "view": 1.0,
    "query_search": 2.0,
    "click": 3.0,
    "compare": 4.0,
}

def load_implicit_feedback(path: str) -> pd.DataFrame:
    frame = pd.read_csv(path)
    frame["Strength"] = frame["Event"].map(EVENT_STRENGTH).astype(np.float32)
    return frame

def build_item_user_matrix(frame: pd.DataFrame) -> tuple[csr_matrix, np.ndarray, np.ndarray]:
    user_codes, user_index = pd.factorize(frame["UserID"], sort=True)
    item_codes, item_index = pd.factorize(frame["ProductCode"], sort=True)

    matrix = coo_matrix(
        (frame["Strength"].astype(np.float32), (item_codes, user_codes)),
        shape=(len(item_index), len(user_index)),
    )

    return matrix.tocsr(), user_index, item_index


Load and preprocess data.

In [24]:
implicit_df = load_implicit_feedback("data/feedback_implicit.csv")
user_item_matrix, user_index, item_index = build_item_user_matrix(implicit_df)
print(f"Matrix shape (items x users): {user_item_matrix.shape}")


Matrix shape (items x users): (34797, 5000)


## 3. Model Training

Split the `user_item` matrix into train and test sets using *leave-1-out* evaluation strategy.

In [25]:
user_items = bm25_weight(user_item_matrix, K1=100, B=0.8).T.tocsr()
train_l1o, test_l1o = leave_k_out_split(user_items, random_state=RANDOM_SEED)
print(f"Leave-1-out | train: {train_l1o.nnz:,} | test: {test_l1o.nnz:,}")


Leave-1-out | train: 89,895 | test: 5,000


Train the ALS recommender using the `train_l1o` train set.

In [26]:
model = AlternatingLeastSquares(factors=64, regularization=0.05, alpha=20.0, random_state=RANDOM_SEED)
model.fit(train_l1o)


  0%|          | 0/15 [00:00<?, ?it/s]

Generate and visualise top-10 recommendations for the first user.

In [27]:
userid = 0
ids, scores = model.recommend(
    userid,
    user_items=None,
    N=10,
    filter_already_liked_items=False,
)
display(pd.DataFrame({
    "item": item_index[ids],
    "score": scores,
    "already_liked": np.in1d(ids, user_items[userid].indices)
}))


,item,score,already_liked
0,0131044121328,0.925285,True
1,0638102660350,0.922950,True
2,8718907133302,0.919939,True
3,0788434108096,0.909490,True
4,0857889007022,0.905493,True
5,5400119540650,0.893326,True
6,3512660038103,0.883225,True
7,7310500178103,0.879426,True
8,0073504200641,0.879410,True
9,0021908130439,0.879386,True


## 4. Model Evaluation

Define functions for recommender evaluation.

In [28]:
def precision_at_k(recommended: list[int], relevant: list[int], k: int):
    hits = sum(1 for item in recommended[:k] if item in relevant)
    denom = min(len(relevant), k)
    return hits / denom

def average_precision_at_k(recommended: list[int], relevant: set[int], k: int) -> float:
    hits = 0
    score = 0.0
    for rank, item_id in enumerate(recommended[:k], start=1):
        if item_id in relevant:
            hits += 1
        score += hits / rank
    denom = min(len(relevant), k)
    return score / denom if denom else 0.0

def mean_average_precision_at_k(
    model: AlternatingLeastSquares,
    train_user_items: csr_matrix,
    test_user_items: csr_matrix,
    k: int,
    *,
    eval_mode: Literal["test", "train"] = "test",
    show_progress: bool = True,
) -> float:

    ap_scores: list[float] = []

    user_items = test_user_items if eval_mode == "test" else train_user_items

    user_iter = trange(user_items.shape[0], desc="Computing MAP@K", leave=True) if show_progress else range(user_items.shape[0])
    for user_idx in user_iter:
        if not (relevant := set(user_items[user_idx].indices.tolist())):
            continue

        rec_ids, _ = model.recommend(
            userid=user_idx,
            user_items=train_user_items[user_idx] if eval_mode == "test" else None,
            N=k,
            filter_already_liked_items=eval_mode == "test",
        )

        ap_scores.append(average_precision_at_k(rec_ids, relevant, k))

    return np.mean(ap_scores) if ap_scores else 0.0


Measure MAP@10 for train set.

In [29]:
map_at_k = mean_average_precision_at_k(
    model=model,
    train_user_items=train_l1o,
    test_user_items=test_l1o,
    eval_mode="train",
    k=10,
)
print(f"MAP@10 (Train): {map_at_k:.4f}")


Computing MAP@K: 100%|██████████| 5000/5000 [00:01<00:00, 2679.45it/s]

MAP@10 (Train): 0.9136


Measure MAP@10 for test set.

In [30]:
map_at_k = mean_average_precision_at_k(
    model=model,
    train_user_items=train_l1o,
    test_user_items=test_l1o,
    eval_mode="test",
    k=10,
)
print(f"MAP@10 (Test): {map_at_k:.4f}")

Computing MAP@K: 100%|██████████| 5000/5000 [00:01<00:00, 3321.89it/s]

MAP@10 (Test): 0.1577


## 5. Hyperparameter Tuning

Grid search hyperparameters.

In [31]:
def grid_search_als(
    param_grid: dict,
    k: int = 10,
) -> pd.DataFrame:
    """Grid search for ALS hyperparameters."""
    results = []
    param_combos = list(product(
        param_grid["factors"],
        param_grid["regularization"],
        param_grid["alpha"],
        param_grid["iterations"],
        param_grid["bm25_K1"],
        param_grid["bm25_B"],
    ))
    # Use tqdm from notebook for widget bar
    for factors, reg, alpha, iters, k1, b in tqdm(param_combos, desc="Grid Search", leave=True):
        weighted = bm25_weight(user_item_matrix, K1=k1, B=b).T.tocsr()
        train_l1o, test_l1o = leave_k_out_split(weighted, random_state=RANDOM_SEED)
        model = AlternatingLeastSquares(
            factors=factors,
            regularization=reg,
            alpha=alpha,
            iterations=iters,
            random_state=RANDOM_SEED,
        )
        model.fit(train_l1o, show_progress=False)
        map_score = mean_average_precision_at_k(
            model, train_l1o, test_l1o, k, show_progress=False
        )
        results.append({
            "factors": factors,
            "regularization": reg,
            "alpha": alpha,
            "iterations": iters,
            "bm25_K1": k1,
            "bm25_B": b,
            f"MAP@{k}": map_score,
        })
    return pd.DataFrame(results)

param_grid = {
    "factors": [32, 64],
    "regularization": [0.01, 0.05],
    "alpha": [10.0, 20.0],
    "iterations": [10, 20],
    "bm25_K1": [50, 100],
    "bm25_B": [0.6, 0.8],
}

results_df = grid_search_als(param_grid)


Grid Search: 100%|██████████| 64/64 [02:19<00:00,  2.19s/it]


Display hyperparameters.

In [32]:
results_df = results_df.sort_values("MAP@10", ascending=False)
results_df


,factors,regularization,alpha,iterations,bm25_K1,bm25_B,MAP@10
42,64,0.01,20.0,10,100,0.6,0.178443
58,64,0.05,20.0,10,100,0.6,0.177243
41,64,0.01,20.0,10,50,0.8,0.176482
46,64,0.01,20.0,20,100,0.6,0.175062
40,64,0.01,20.0,10,50,0.6,0.172556
...,...,...,...,...,...,...,...
19,32,0.05,10.0,10,100,0.8,0.070334
17,32,0.05,10.0,10,50,0.8,0.066655
3,32,0.01,10.0,10,100,0.8,0.065458
21,32,0.05,10.0,20,50,0.8,0.065114


## 6. Train the Best Model

Train the final ALS model using the best hyperparameters obtained from the grid search.

In [33]:
top_model_row = results_df.sort_values("MAP@10", ascending=False).iloc[0]

best_factors = top_model_row["factors"]
best_regularization = top_model_row["regularization"]
best_alpha = top_model_row["alpha"]
best_iterations = top_model_row["iterations"]
best_bm25_K1 = top_model_row["bm25_K1"]
best_bm25_B = top_model_row["bm25_B"]

weighted_full_matrix = bm25_weight(user_item_matrix, K1=best_bm25_K1, B=best_bm25_B).T.tocsr()

final_model = AlternatingLeastSquares(
    factors=int(best_factors),
    regularization=best_regularization,
    alpha=best_alpha,
    iterations=int(best_iterations),
    random_state=RANDOM_SEED,
)
final_model.fit(weighted_full_matrix)


  0%|          | 0/10 [00:00<?, ?it/s]

Save the final model to a file using using `pickle` for production use.

In [34]:
model_dir = Path("model")
model_dir.mkdir(exist_ok=True)

with open(model_dir / "implicit_recommender_model.pkl", "wb") as f:
    pickle.dump(final_model, f)
